## 目标
将新闻资讯中提及的股票名称提取出来，并匹配到股票名录数据表。  
完整的步骤：  
1. 数据预处理：首先，预处理新闻资讯和股票名录数据表。对于新闻资讯，通过分词、去除停用词等方式来处理文本。对于股票名录数据表，需要统一格式，去除名称中的空格。
2. 实体识别：接下来，识别出新闻资讯中提及的股票名称。通过命名实体识别（Named Entity Recognition, NER）技术来实现。NER是一种常见的自然语言处理任务，用于识别文本中的特定类型的实体，如人名、地名、组织名等。使用预训练的NER模型。
3. 实体链接：在识别出股票名称后，将这些名称链接到股票名录数据表中的相应股票，通过实体链接（Entity Linking）技术来实现。通过查找股票名称在股票名录数据表中的匹配来实现实体链接。
4. 结果评估：通过准确率、召回率等指标来评估你的实体识别和实体链接的性能。也可以通过人工检查一部分结果来进行评估。

在本次实现中，由于财联社电报中的股票名称较好提取，暂时只用到了简单的文本匹配方案。

In [1]:
import akshare as ak
import pandas as pd
import re # 用于正则表达式
import difflib # 用于名称相似度匹配

In [2]:
stock_info_df= pd.read_csv('data/stock_info.csv')  # 读取本地股票列表
# 数据清洗：去除NaN值，填充为空字符串
stock_info_df = stock_info_df.fillna('') 

# 数据清洗：对'A股简称'、'B股简称'、'H股简称'，都需要去除空格
stock_info_df['A股简称'] = stock_info_df['A股简称'].str.replace(' ', '') # 去除空格
stock_info_df['B股简称'] = stock_info_df['B股简称'].str.replace(' ', '') # 去除空格
stock_info_df['H股简称'] = stock_info_df['H股简称'].str.replace(' ', '') # 去除空格

# 数据清洗：对于stock_info_df的'曾用简称'，采用">>"分隔了多个值，且存在空格，样例为“G*ST华光>> ST华  光>> *ST华光>> ST华光>> *ST华光>> 青鸟华光”。需要去除空格，并对逐个值进行匹配。
stock_info_df['曾用简称'] = stock_info_df['曾用简称'].str.replace(' ', '') # 去除空格
stock_info_df['曾用简称'] = stock_info_df['曾用简称'].str.split('>>') # 将 '>>' 分隔的字符串分割为列表

In [3]:
# 功能：初始化stock_cls_telegram_df，用于存储最新获得的新闻条目。

# 获取最新的财联社-电报，并去重
stock_cls_telegram_df = ak.stock_telegraph_cls().drop_duplicates()
# 对stock_cls_telegram_df按index降序
stock_cls_telegram_df = stock_cls_telegram_df.sort_index(ascending=False)

# 对stock_cls_telegram_df重置索引，否则在使用loc()时会报错
stock_cls_telegram_df = stock_cls_telegram_df.reset_index(drop=True)
# 添加列 '提取证券名称'、'匹配证券代码'
stock_cls_telegram_df['提取证券名称'] = ''
stock_cls_telegram_df['匹配证券代码'] = ''

In [4]:
# 获取财联社-每日快讯。测试结论：返回的数据是过期的，无法使用
# stock_zh_a_alerts_cls_df = ak.stock_zh_a_alerts_cls()
# print(stock_zh_a_alerts_cls_df)
# stock_zh_a_alerts_cls_df.to_csv("stock_zh_a_alerts_cls.csv")

In [4]:
# 从标题中提取证券名称，使用的方法是用正则表达式提取从文本开头至‘：’前的最多6个字符，且排除‘：’前<=2个字符的情况。
def extract_stock_name(text):
    # 检查'：'存在，且'：'前是否有超过2个字符
    if '：' in text and len(text.split('：')[0]) > 2:
        # 提取从文本开头至'：'前的最多6个字符
        match = re.match(r'^.{1,6}', text.split('：')[0])
        if match:
            return match.group(0)
    # 如果不满足长度要求，返回空字符串
    return ''

for i in range(len(stock_cls_telegram_df)):
    stock_cls_telegram_df.loc[i, '提取证券名称'] = extract_stock_name(stock_cls_telegram_df.loc[i, '标题'])

# 测试函数
# texts = ['中公教育：预计2023年半年度净利润7200万元-9200万元 同比扭亏为盈', '中广核矿业：二季度生产天然铀676.7tU 计划完成率为105.8%', 'ST浪潮：后面的文本', 'S*ST浪潮：后面的文本','京东方A：预计上半年净利润同比下降88%-89%','华电国际电力股份：上半年预计盈利约23.5亿元至28.2亿元 同比增43%到71%','士兰微：预计上半年净亏损5037万元左右','收评：沪指窄幅震荡微涨 AI概念股持续反弹','机构：到2030年 边缘数据中心市场价值将达到452亿美元','龙虎榜|中国联通今日涨停 沪股通买入3.55亿元并卖出9274.75万元','国防部回应美称中国欲成为亚洲霸主：中国的目标是做更好更强的自己']
# for text in texts:
#     print(f'"{text}" -> "{extract_stock_name(text)}"')

# 测试结论：含少量错误，基本可用。也有一些在内容中提及的股票没有提取，需要用到LLM的方法。


In [5]:
# 功能：实体连接，stock_cls_telegram_df通过'证券简称'匹配出'匹配正确代码'。
# 优先使用stock_info_df的'A股简称'、'B股简称'、'H股简称'，其次是'曾用简称'（是一个列表，包含多个名称），来匹配 stock_cls_telegram_df的'提取证券名称'。得到stock_info_df匹配记录的'证券完整代码'，写入stock_cls_telegram_df的'匹配证券代码'列。
# 由于存在 历史名称 和 当前简称 重复的情况，故制定以上匹配优先级。例如用“金冠电气”匹配到两只股票：300510.SZ 金冠股份（曾名 金冠电气）、688517.SH 金冠电气，此时应认为匹配结果是当前名称的 688517.SH 金冠电气。
# 每条新闻仅输出一个匹配结果。如果存在多条结果，则取相似度最高的记录。步骤为：1、先找出stock_info_df存在相似的记录；2、对每条记录的'A股简称'、'B股简称'、'H股简称'、'曾用简称'统计关键词出现次数；3、去关键词出现次数最多的那条记录。


# 遍历每条新闻，尝试匹配股票名称
for i in range(len(stock_cls_telegram_df)):
    stock_name = stock_cls_telegram_df.loc[i, '提取证券名称']
    match_found = False

    if stock_name != '':  # 仅处理'提取证券名称'不为空的记录
        # 优先使用 'A股简称'、'B股简称'、'H股简称' 进行匹配
        for column_name in ['A股简称', 'B股简称', 'H股简称']:
            if stock_info_df.iloc[i][column_name] == stock_name: # 如果该列中的股票名称匹配，取第一个匹配结果（因为每条新闻仅输出一个匹配结果）
                matche = stock_info_df[i] # 使用matche存储匹配的行
                stock_cls_telegram_df.loc[i, '匹配证券代码'] = matche.iloc[0]['证券完整代码']
                match_found = True
                break

        if not match_found:
            # 如果没有找到匹配，使用 '曾用简称' 进行匹配
            for old_names in stock_info_df['曾用简称']:
                if stock_name == old_names:
                    matche = stock_info_df[i]
                    # 如果找到匹配，取第一个匹配结果（因为每条新闻仅输出一个匹配结果）
                    stock_cls_telegram_df.loc[i, '匹配证券代码'] = matche.iloc[0]['证券完整代码']
                    match_found = True
                    break

        if not match_found:
            # 如果仍然没有找到匹配，计算按相似度并选择相似度最高的结果
            # 步骤为：1、先找出stock_info_df存在相似的记录；2、对每条记录的'A股简称'、'B股简称'、'H股简称'、'曾用简称'统计关键词出现次数；3、去关键词出现次数最多的那条记录。
            matches = stock_info_df['A股简称'].tolist() + stock_info_df['B股简称'].tolist() + stock_info_df['H股简称'].tolist() + stock_info_df['曾用简称'].tolist()
            matches = [str(i) for i in matches]  # 确保所有元素都是字符串
            similarity = difflib.get_close_matches(stock_name, matches, n=1, cutoff=0.8)
            if similarity:
                match = stock_info_df[(stock_info_df['A股简称'] == similarity[0]) | (stock_info_df['B股简称'] == similarity[0]) | (stock_info_df['H股简称'] == similarity[0]) | (stock_info_df['曾用简称'] == similarity[0])]
                stock_cls_telegram_df.loc[i, '匹配证券代码'] = match.iloc[0]['证券完整代码']
                
# 测试结论：通过。虽然模糊匹配的实现没有完全按备注的要求，但是基本可用。

In [6]:
# 将最新的财联社-电报，与本地文件stock_cls_telegram_df进行比对，如果有新的电报，则将新的电报写入本地文件。


try: # 如果存在本地文件则先读取
    stock_cls_telegram_old_df = pd.read_csv('data/stock_cls_telegram_df.csv')  # 每次启动，如果存在本地文件则先读取
    # 与本地文件stock_cls_telegram_df进行比对，重复的标准是：含有相同的'发布时间'+‘发布日期’+‘标题’+'内容'。如果有新的电报，则将新的电报写入本地文件。
    stock_cls_telegram_df = stock_cls_telegram_df.append(stock_cls_telegram_old_df).drop_duplicates(subset=['发布时间','发布日期','标题','内容'],keep='last')

    stock_cls_telegram_df.to_csv("data/stock_cls_telegram.csv",index=False, encoding='utf-8-sig')
except:  # 如果本地文件不存在，则直接写入
    stock_cls_telegram_df.to_csv("data/stock_cls_telegram.csv",index=False, encoding='utf-8-sig')

